# XGBoost

In [1]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization
from IPython.display import display
from ipywidgets import IntProgress
from sklearn import metrics
import xgboost as xgb
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')
hiperparametros_csv = os.path.join(current_folder, 'hiperparametros', 'xgboost.csv')

Cargo el df con los features.

In [3]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [4]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

Cargo los sets de entrenamiento, testing y predicción.

In [5]:
%run $set_de_entrenamiento_testing_y_prediccion

labels_with_features = labels.merge(df_features, how='inner', on='person')
data = labels_with_features.drop('label', axis=1)
target = labels_with_features['label']

## Entrenamiento rápido

Con cross validation de xgboost. Lo bueno de esto es que al final me da el *num_boost_round* óptimo.

In [53]:
param = {
    'silent': 1,
    'objective': 'reg:logistic',
    'alpha': 3.845311207046479,
    'colsample_bylevel': 0.6605668347627213,
    'colsample_bytree': 0.5279014819087092,
    'eta': 0.15803667962605694,
    'gamma': 6.219264874528072,
    'lambda': 1.1181195507921775,
    'max_delta_step': 7.592652591386328,
    'max_depth': 9,
    'min_child_weight': 4.302125582335056,
    'subsample': 0.43744176565530823
}
cv = 10 # cantidad de splits en el cross validation
num_round = 100 # cantidad de veces que se boostea

In [54]:
%%time
dtrain = xgb.DMatrix(data, label=target)
result = xgb.cv(param, dtrain, nfold=cv, metrics='auc', verbose_eval=False, shuffle=False, stratified=False, num_boost_round=num_round)

CPU times: user 1min 27s, sys: 2.48 s, total: 1min 29s
Wall time: 1min 14s


El índice + 1 es el *num_boost_round* óptimo. 

In [55]:
result.loc[[result['test-auc-mean'].idxmax()]]

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
64,0.878447,0.011276,0.902251,0.001519


## Búsqueda de hiperparámetros con Grid Search

Vamos a hacer un kfold con sklearn.

**Nota**: está busqueda ya no es óptima, es mejor realizarla usando GridSearchCV de sklearn o Bayesian Optimization que está más abajo.

In [9]:
splits = 2
max_depth_values = 2
eta_values = 2
gamma_values = 2
num_round_values = 2

In [10]:
param = {
    'max_depth': 30,
    'eta': 0,
    'gamma': 0,
    'silent': 1,
    'objective': 'binary:logistic',
    'nthread': 10,
    'eval_metric': 'auc'
}

In [11]:
def calculate_auc():
    return metrics.roc_auc_score(labels_test['label'], labels_test['label_predicted'])

In [12]:
labels_with_features = labels.merge(df_features, how='inner', on='person')

In [13]:
columns = list(labels_with_features.columns)
columns.remove('label')

In [14]:
f = IntProgress(min=0, max=splits*max_depth_values*eta_values*gamma_values*num_round_values)
display(f) # display the bar

kf = KFold(n_splits=splits, shuffle=False)
results = pd.DataFrame(columns=['k', 'max_depth', 'eta', 'gamma', 'num_round', 'auc'])
index = 0
k = 0
for train_index, test_index in kf.split(labels):
    
    labels_training = labels_with_features.iloc[train_index]
    labels_test = labels_with_features.iloc[test_index]
    assert(labels_training.merge(labels_test, how='inner', on='person').shape[0] == 0)
    train_matrix = xgb.DMatrix(labels_training.loc[:, columns], label=labels_training['label'])
    test_matrix = xgb.DMatrix(labels_test.loc[:, columns])
    
    for max_depth, eta, gamma, num_round in np.ndindex((max_depth_values, eta_values, gamma_values, num_round_values)):
        eta=eta/eta_values
        param['max_depth'] = max_depth
        param['eta'] = eta
        param['gamma'] = gamma
        
        bst = xgb.train(param, train_matrix, num_round)
        labels_test['label_predicted'] = bst.predict(test_matrix)
        
        results.loc[index] = k, max_depth, eta, gamma, num_round, calculate_auc()
        
        index+=1
        f.value += 1
    
    k += 1

IntProgress(value=0, max=32)

Promedio los hiperparámetros.

In [15]:
results_mean = results.groupby(['max_depth', 'eta', 'gamma', 'num_round'])[['auc']].mean()
mejor_resultado = results_mean.loc[results_mean.idxmax()]
mejor_resultado

,,,,auc
max_depth,eta,gamma,num_round,
1.0,0.5,0.0,1.0,0.776814


Escribo los nuevos resultados en un archivo.

In [16]:
params = mejor_resultado.reset_index().to_dict('records')[0]
auc = params.pop('auc')

In [17]:
hyperparameter_data = {
    'algorithm': 'xgboost',
    'hyperparameters': params,
    'cv_splits': splits,
    'auc': auc,
    'features': data.columns
}

In [18]:
%run -i write_hyperparameters.py

## Hiperparámetros con Bayesian Optimization

In [48]:
pbounds = {
    'max_depth': (2, 30),
    'eta': (0, 1),
    'gamma': (0, 20),
    'min_child_weight': (1, 8),
    'max_delta_step': (1, 8),
    'subsample': (0, 1),
    'colsample_bytree': (0, 1),
    'colsample_bylevel': (0, 1),
    'lambda': (1, 10),
    'alpha': (0, 8)
}

discrete = ['max_depth'] # parámetros discretos
cv_splits = 10 # cantidad de splits en el cv
num_round = 100 # cantidad máxima de boosts

Falta optimizar otros parámetros discretos:
 - booster 
 - min_child_weight 
 - max_delta_step 
 - etc...

In [49]:
dtrain = xgb.DMatrix(data, label=target)
def cv_score_xgb(**param):
    param['silent'] = 1
    param['objective'] = 'reg:logistic'
#     param['scale_pos_weight'] = 19
    
    # transformo los valores que deben ser discretos
    for d in discrete:
        param[d] = int(param[d])
    
    # hago el cv
    scores = xgb.cv(param, dtrain, nfold=cv_splits, metrics='auc', verbose_eval=False, shuffle=False, stratified=False, num_boost_round=num_round, early_stopping_rounds=20)
    return scores['test-auc-mean'].max()

In [64]:
%%time
optimizer = BayesianOptimization(f=cv_score_xgb, pbounds=pbounds)
optimizer.probe(
    params = {
        'alpha': 3.845311207046479,
        'colsample_bylevel': 0.6605668347627213,
        'colsample_bytree': 0.5279014819087092,
        'eta': 0.15803667962605694,
        'gamma': 6.219264874528072,
        'lambda': 1.1181195507921775,
        'max_delta_step': 7.592652591386328,
        'max_depth': 9,
        'min_child_weight': 4.302125582335056,
        'subsample': 0.43744176565530823
    }
)
    
optimizer.maximize(
    init_points=0,
    n_iter=100,
)

|   iter    |  target   |   alpha   | colsam... | colsam... |    eta    |   gamma   |  lambda   | max_de... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.848    |  3.845    |  0.6606   |  0.5279   |  0.158    |  6.219    |  1.118    |  7.593    |  9.0      |  4.302    |  0.4374   |
|  2        |  0.8688   |  6.801    |  0.8263   |  0.582    |  0.5256   |  13.61    |  3.883    |  6.652    |  15.51    |  2.052    |  0.8114   |
|  3        |  0.774    |  0.0      |  0.0      |  0.9725   |  0.9967   |  0.0      |  10.0     |  1.0      |  30.0     |  8.0      |  1.0      |
|  4        |  0.5      |  0.0      |  0.0      |  8.662e-1 |  1.0      |  20.0     |  10.0     |  1.0      |  2.0      |  8.0      |  0.0      |
|  5        |  0.5      |  0.0      |  1.0      |  0.0      |  0.0      |  20.0     |  1.0      |  8.0      |  30.0     |  1

|  34       |  0.5      |  0.0      |  1.0      |  1.0      |  0.0      |  5.822    |  1.0      |  1.0      |  30.0     |  8.0      |  1.0      |
|  35       |  0.5      |  8.0      |  1.0      |  1.0      |  0.0      |  0.0      |  10.0     |  8.0      |  30.0     |  1.0      |  1.0      |
|  36       |  0.8482   |  0.0      |  1.0      |  1.0      |  1.0      |  0.0      |  1.0      |  8.0      |  9.436    |  8.0      |  1.0      |
|  37       |  0.7634   |  8.0      |  0.0      |  1.0      |  1.0      |  20.0     |  1.0      |  1.0      |  30.0     |  1.0      |  1.0      |
|  38       |  0.5      |  8.0      |  1.0      |  0.0      |  0.0      |  20.0     |  10.0     |  1.0      |  30.0     |  1.0      |  1.0      |
|  39       |  0.7458   |  8.0      |  0.0      |  1.0      |  1.0      |  10.77    |  10.0     |  1.0      |  17.91    |  8.0      |  1.0      |
|  40       |  0.7208   |  8.0      |  0.0      |  1.0      |  1.0      |  20.0     |  10.0     |  8.0      |  30.0     |  8

|  68       |  0.8664   |  8.0      |  1.0      |  1.0      |  1.0      |  4.471    |  1.0      |  1.0      |  11.9     |  8.0      |  1.0      |
|  69       |  0.8562   |  8.0      |  1.0      |  1.0      |  1.0      |  0.0      |  1.0      |  8.0      |  10.82    |  1.0      |  1.0      |
|  70       |  0.8645   |  0.0      |  1.0      |  1.0      |  1.0      |  20.0     |  1.0      |  8.0      |  12.33    |  8.0      |  1.0      |
|  71       |  0.8666   |  0.0      |  1.0      |  1.0      |  1.0      |  13.36    |  1.0      |  1.0      |  15.19    |  8.0      |  1.0      |
|  72       |  0.8619   |  8.0      |  1.0      |  1.0      |  1.0      |  20.0     |  1.0      |  8.0      |  18.66    |  1.0      |  1.0      |
|  73       |  0.862    |  8.0      |  1.0      |  1.0      |  1.0      |  0.0      |  10.0     |  1.0      |  30.0     |  8.0      |  1.0      |
|  74       |  0.8698   |  8.0      |  1.0      |  1.0      |  1.0      |  11.06    |  10.0     |  1.0      |  30.0     |  8

In [34]:
optimizer.max

{'params': {'alpha': 3.845311207046479,
  'colsample_bylevel': 0.6605668347627213,
  'colsample_bytree': 0.5279014819087092,
  'eta': 0.15803667962605694,
  'gamma': 6.219264874528072,
  'lambda': 1.1181195507921775,
  'max_delta_step': 7.592652591386328,
  'max_depth': 9.93517044684219,
  'min_child_weight': 4.302125582335056,
  'subsample': 0.43744176565530823},
 'target': 0.8784471}

Guardo el resultado en un archivo.

In [35]:
params = optimizer.max['params'].copy()
params['max_depth'] = int(params['max_depth'])
params['silent'] = 1
result = xgb.cv(params, dtrain, nfold=cv_splits, metrics='auc', verbose_eval=False, shuffle=False, stratified=False, num_boost_round=num_round)
params['num_round'] = result['test-auc-mean'].idxmax() + 1
del params['silent']

In [36]:
hyperparameter_data = {
    'algorithm': 'xgboost',
    'hyperparameters': params,
    'cv_splits': cv_splits,
    'auc': optimizer.max['target'],
    'features': data.columns
} 

In [37]:
%run -i write_hyperparameters.py

## Predecir labels desconocidos

In [56]:
dtrain = xgb.DMatrix(data, label=target)

In [57]:
param = {
    'silent': 1,
    'objective': 'reg:logistic',
    'alpha': 3.845311207046479,
    'colsample_bylevel': 0.6605668347627213,
    'colsample_bytree': 0.5279014819087092,
    'eta': 0.15803667962605694,
    'gamma': 6.219264874528072,
    'lambda': 1.1181195507921775,
    'max_delta_step': 7.592652591386328,
    'max_depth': 9,
    'min_child_weight': 4.302125582335056,
    'subsample': 0.43744176565530823 
}
num_round = 10

In [58]:
bst = xgb.train(param, dtrain, num_boost_round=65)

Predigo:

In [59]:
labels_to_predict_with_features = labels_to_predict.merge(df_features, how='inner', on='person')

In [60]:
assert(labels_to_predict.shape[0] == labels_to_predict_with_features.shape[0])

In [61]:
matrix = xgb.DMatrix(labels_to_predict_with_features.loc[:, columns])

In [62]:
labels_to_predict['label'] = bst.predict(matrix)

In [63]:
labels_to_predict.to_csv(predicciones_csv)